In [ ]:
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
#for one way ANOVA:
#from scipy.stats import f_oneway
#for three way ANOVA:
#import statsmodels.api as sm
#from statsmodels.formula.api import ols

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = '/content/drive/MyDrive/Data Science/Project1: Math 3439/data/'
df_sales = pd.read_csv(data_path+'sales.csv')
df_stores = pd.read_csv(data_path+'stores.csv')
df_store_features = pd.read_csv(data_path+'store_features.csv')

In [ ]:
#converting dates in sales and store features to datetime
df_store_features['Date'] = pd.to_datetime(df_store_features['Date'])
df_sales['Date'] = pd.to_datetime(df_sales['Date'])
#removing dept from sales and adjusting sales amounts, based on the avg amount for that date
df_sales = df_sales.groupby(['Store', 'Date', 'IsHoliday']).agg({'Weekly_Sales': np.sum}).reset_index()
df_sales_mean = df_sales.groupby('Date').agg({'Weekly_Sales': 'mean'}).reset_index()
df_sales_mean.columns = ['Date', 'Sales_Avg_For_Date']
df_sales = pd.merge(df_sales, df_sales_mean, on='Date')
df_sales['Weekly_Sales_Adjusted'] = df_sales['Weekly_Sales'] - df_sales['Sales_Avg_For_Date']

In [ ]:
#merging dataframes
df_merged = pd.merge(df_sales, df_stores, on='Store')
df_merged.head()
df_merged = pd.merge(df_merged, df_store_features, on=['Store', 'Date', 'IsHoliday'])
df_merged.head()
df_merged.shape

(2160, 17)

In [ ]:
#adding 'bins' to unemployment and fuel prices in order to use them as categorical variables for anova
#min unemployment is 5%, max is 14%
bins_unemp = np.linspace(5,14, num = 10)
df_merged['bin_unemp'] = pd.cut(df_merged['Unemployment'], bins_unemp)
#min fuel price is about 2.4, max is about 3.4
bins_fuel = np.linspace(2.4, 3.4, num=11)
df_merged['bin_fuel'] = pd.cut(df_merged['Fuel_Price'], bins_fuel)
df_merged.head()

In [ ]:
#writing merged dataframe to csv
df_merged.to_csv(data_path + 'merged_datasets_adj_bins.csv', encoding='utf-8', index=False)